In [1]:
# systems tools
import os
import shutil
import sys
import random
import time
import sys
import signal

# multiprocess
import threading
#import psutil

#format
import string
import json

In [2]:
#configuration for the experimenal setup (what stays unchanged)
def configuration_set():
    config = {}
    #paths
    config['path_initial'] = os.getcwd()
    print ("Inital path: {0}".format(config['path_initial']))    
    config['path_results'] = config['path_initial']+ "/logs"
        
    
    # Metadata for experiments
    config['user']="theoleyr"
    config['exp_duration']=180        # for the iot lab reservation (collection of runs), in minutes
    #config['subexp_duration']        # no timeout
    config['exp_resume']=True
    config['exp_name']="owsnccajup"

    # Parameters of the experiment
    config['board']="iot-lab_M3"
    config['toolchain']="armgcc"
    config['archi']="m3"
    config['site']="strasbourg"    

    #Parameters for this set of experiments
    # list of motes
    config['nodes_list']=[ 60 , 64 ]
    config['dagroots_list']=[ 43 ]

    #parameters for the code
    config['badmaxrssi'] = 100
    config['goodminrssi'] = 100
    config['lowestrankfirst'] = 1
    config['anycast'] = 1
    
    #openvisualizer configuration (output + 2 inputs)
    config['conf_file']= config['path_initial'] + "/logs/logging.conf"
    config['conf_file_start']= config['path_initial'] + "/../loggers/logging_start.conf"
    config['conf_file_end']= config['path_initial'] + "/../loggers/logging_end.conf"
    
    # openvisualizer directory
    config['code_sw_src'] = config['path_initial'] + "/../../openvisualizer/"
    if (os.path.exists(config['code_sw_src']) == False):
        print("{0} does not exist".format(config['code_sw_src']))
        exit(-4)
    config['code_sw_gitversion']="e039a05"

    # firmware part
    config['code_fw_src']= config['path_initial'] + "/../../openwsn-fw/"
    if (os.path.exists(config['code_fw_src']) == False):
        print("{0} does not exist".format(config['code_fw_src']))
        exit(-4)
    config['code_fw_gitversion']="515eafa7"
    config['code_fw_bin']=config['code_fw_src']+"build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog"
    
    #Only in simulation mode!
    if (config['board'] == "python"):
        config['nbnodes']="5"
        config['topology']="---load-topology " + config['path_initial'] + "topologies/topology-3nodes.json"

    return(config)

In [3]:
config = configuration_set()

#custom libraries
sys.path.insert(0, config['path_initial']+ "/../")
import iotlabowsn

Inital path: /home/theoleyre/openwsn/scripts/jupyter


In [ ]:
# killall the children (openweb + openvisualizer)

iotlabowsn.run_command("sudo killall python")
iotlabowsn.run_command("sudo killall /bin/sh")

print("Clean start: all the OV and openweb server have been killed")

In [4]:
#prints the headers of a section
def print_header(msg):
    print("\n\n---------------------------------------------")
    print("     " + msg)
    print("---------------------------------------------\n\n")


In [5]:
# ----- INIT

print_header("Initialization")
#config = configuration_set()
iotlabowsn.openvisualizer_install(config)



---------------------------------------------
     Initialization
---------------------------------------------


Install the current version of Openvisualizer
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.

Installation ok


In [6]:
# ---- RESERVATION /RESUME EXPERIMENT ----
print_header("Reservation (experiment)")
if ( config['exp_resume'] == True):
    exp_id = iotlabowsn.get_running_id(config);
if exp_id is not None:
    print("Resume the experiment id {0}".format(exp_id))
else:
    exp_id = iotlabowsn.reserve(config)
iotlabowsn.wait_running(exp_id)




---------------------------------------------
     Reservation (experiment)
---------------------------------------------


Verification that the list of nodes is correct for the exp_id 264032
  -> m3-43.strasbourg.iot-lab.info
     43 is a dagroot (in [43])
  -> m3-64.strasbourg.iot-lab.info
     64 is a node (in [60, 64])
  -> m3-60.strasbourg.iot-lab.info
     60 is a node (in [60, 64])
Resume the experiment id 264032
iotlab-experiment wait -i 264032
"Running"



In [7]:
# ---- COMPIL + FLASHING----

print_header("Compilation")
iotlabowsn.compilation_firmware(config)

print_header("Flashing")
iotlabowsn.flashing_motes(exp_id, config)




---------------------------------------------
     Compilation
---------------------------------------------


scons board=iot-lab_M3 toolchain=armgcc  boardopt=printf modules=coap,udp apps=cjoin,cexample debugopt=CCA,schedule  scheduleopt=anycast,lowestrankfirst  stackcfg=badmaxrssi:100,goodminrssi:100  oos_openwsn 
OUT= scons: Reading SConscript files ...

 ___                 _ _ _  ___  _ _ 
| . | ___  ___ ._ _ | | | |/ __>| \ |
| | || . \/ ._>| ' || | | |\__ \|   |
`___'|  _/\___.|_|_||__/_/ <___/|_\_|
     |_|                  openwsn.org

scons: done reading SConscript files.
scons: Building targets ...
arm-none-eabi-size --format=berkeley -d --totals build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog
   text	   data	    bss	    dec	    hex	filename
 146348	    208	  65328	 211884	  33bac	build/iot-lab_M3_armgcc/projects/common/03oos_openwsn_prog
 146348	    208	  65328	 211884	  33bac	(TOTALS)
scons: done building targets.

ERR= 


------------------------------------

In [8]:
# ---- OpenVisualizer ----

print_header("Openvizualiser")
iotlabowsn.openvisualizer_create_conf_file(config)
t_openvisualizer = iotlabowsn.openvisualizer_start(config)



---------------------------------------------
     Openvizualiser
---------------------------------------------


python /usr/local/bin/openv-server --opentun --wireshark-debug --mqtt-broker 127.0.0.1 -d --fw-path /home/theoleyre/openwsn/openwsn-fw --lconf /home/theoleyre/openwsn/scripts/jupyter/logs/logging.conf --iotlab-motes m3-43.strasbourg.iot-lab.info m3-60.strasbourg.iot-lab.info m3-64.strasbourg.iot-lab.info 
No running openvisualizer process
Running openvisualizer in a separated process
Thread <Thread(Thread-4, started 140700326795008)> started
Openvisualizer seems correctly running


In [9]:
# ---- Openweb server (optional, for debuging via a web interface) ----

print_header("Openweb server")
t_openwebserver = iotlabowsn.openwebserver_start(config)



---------------------------------------------
     Openweb server
---------------------------------------------


python /usr/local/bin/openv-client view web --debug ERROR
Running openweb server in a separated thread
Thread <Thread(Thread-5, started 140699974498048)> started, pid 24077


In [10]:
# ---- Boots the motes ----

print_header("Configure Motes")
iotlabowsn.mote_boot(exp_id)
iotlabowsn.dagroot_set(config)



---------------------------------------------
     Configure Motes
---------------------------------------------


iotlab-node --reset -i 264032
OUT= {
    "0": [
        "m3-43.strasbourg.iot-lab.info", 
        "m3-60.strasbourg.iot-lab.info", 
        "m3-64.strasbourg.iot-lab.info"
    ]
}

ERR= 
openv-client root m3-43.strasbourg.iot-lab.info
OUT= Ok!

ERR= 


In [11]:
# ---- Exp running ----

print_header("Execution")
print("gpid me: {0}".format(os.getpgid(os.getpid())))
print("pid me: {0}".format(os.getpid()))
print("nb threads = {0}".format(threading.active_count()))




---------------------------------------------
     Execution
---------------------------------------------


gpid me: 24077
pid me: 24077
nb threads = 7


In [ ]:
# killall the children (openweb + openvisualizer)

iotlabowsn.run_command("sudo killall python")
iotlabowsn.run_command("sudo killall /bin/sh")

print("Clean end: all the processes have been killed")